## Дослідження кластеризації методом K-Means за допомогою R та принципів Tidy Data.

### [**Тест перед лекцією**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

У цьому уроці ви навчитеся створювати кластери за допомогою пакету Tidymodels та інших пакетів екосистеми R (назвемо їх друзями 🧑‍🤝‍🧑), а також використовуючи набір даних про нігерійську музику, який ви імпортували раніше. Ми розглянемо основи методу K-Means для кластеризації. Пам’ятайте, що, як ви дізналися в попередньому уроці, існує багато способів роботи з кластерами, і метод, який ви використовуєте, залежить від ваших даних. Ми спробуємо метод K-Means, оскільки це найпоширеніша техніка кластеризації. Почнемо!

Терміни, які ви вивчите:

-   Оцінка силуету

-   Метод ліктя

-   Інерція

-   Дисперсія

### **Вступ**

[Кластеризація методом K-Means](https://wikipedia.org/wiki/K-means_clustering) — це метод, що походить із галузі обробки сигналів. Він використовується для поділу та групування даних у `k кластерів` на основі схожості їхніх характеристик.

Кластери можна візуалізувати як [діаграми Вороного](https://wikipedia.org/wiki/Voronoi_diagram), які включають точку (або "насіння") та її відповідну область.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Інфографіка від Jen Looper</figcaption>

Кластеризація методом K-Means включає наступні кроки:

1.  Дата-сайєнтист починає з визначення бажаної кількості кластерів, які потрібно створити.

2.  Далі алгоритм випадково вибирає K спостережень із набору даних, які слугуватимуть початковими центрами кластерів (тобто центроїдами).

3.  Потім кожне з решти спостережень призначається до найближчого центроїда.

4.  Далі обчислюється нове середнє кожного кластеру, і центроїд переміщується до цього середнього.

5.  Тепер, коли центри були перераховані, кожне спостереження знову перевіряється, щоб визначити, чи може воно бути ближчим до іншого кластеру. Усі об’єкти знову перепризначаються, використовуючи оновлені середні кластерів. Кроки перепризначення кластерів і оновлення центроїдів повторюються ітеративно, доки призначення кластерів перестануть змінюватися (тобто, коли досягнуто збіжності). Зазвичай алгоритм завершується, коли кожна нова ітерація призводить до незначного переміщення центроїдів, і кластери стають статичними.

<div>

> Зверніть увагу, що через рандомізацію початкових k спостережень, які використовуються як стартові центроїди, ми можемо отримувати трохи різні результати кожного разу, коли застосовуємо процедуру. З цієї причини більшість алгоритмів використовують кілька *випадкових стартів* і вибирають ітерацію з найменшим WCSS. Таким чином, настійно рекомендується завжди запускати K-Means із кількома значеннями *nstart*, щоб уникнути *небажаного локального оптимуму.*

</div>

Ця коротка анімація, створена за допомогою [ілюстрацій](https://github.com/allisonhorst/stats-illustrations) Еллісон Хорст, пояснює процес кластеризації:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Ілюстрація від @allison_horst</figcaption>

Фундаментальне питання, яке виникає під час кластеризації, таке: як визначити, на скільки кластерів розділити ваші дані? Одним із недоліків використання методу K-Means є те, що вам потрібно встановити `k`, тобто кількість `центроїдів`. На щастя, `метод ліктя` допомагає оцінити гарне початкове значення для `k`. Ви спробуєте це за хвилину.

### 

**Передумови**

Ми продовжимо з того місця, де зупинилися в [попередньому уроці](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), де ми аналізували набір даних, створювали багато візуалізацій і фільтрували набір даних до цікавих спостережень. Обов’язково перегляньте його!

Нам знадобляться деякі пакети для виконання цього модуля. Ви можете встановити їх за допомогою: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Альтернативно, скрипт нижче перевіряє, чи є у вас необхідні пакети для завершення цього модуля, і встановлює їх, якщо деякі відсутні.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Давайте розпочнемо!

## 1. Танці з даними: Визначимо 3 найпопулярніші музичні жанри

Це короткий огляд того, що ми робили на попередньому уроці. Давайте розберемо дані!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 Це пройшло чудово!

## 2. Більше дослідження даних.

Наскільки чисті ці дані? Давайте перевіримо наявність викидів за допомогою коробкових діаграм. Ми зосередимося на числових стовпцях із меншою кількістю викидів (хоча ви можете видалити викиди). Коробкові діаграми можуть показати діапазон даних і допоможуть обрати, які стовпці використовувати. Зверніть увагу, що коробкові діаграми не показують дисперсію, яка є важливим елементом для якісних кластеризованих даних. Будь ласка, ознайомтеся з [цією дискусією](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) для подальшого читання.

[Коробкові діаграми](https://en.wikipedia.org/wiki/Box_plot) використовуються для графічного зображення розподілу `числових` даних, тому давайте почнемо з *вибору* всіх числових стовпців разом із популярними музичними жанрами.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Подивіться, як селектор `where` робить це простим 💁? Досліджуйте інші подібні функції [тут](https://tidyselect.r-lib.org/).

Оскільки ми будемо створювати коробкові діаграми для кожної числової ознаки і хочемо уникнути використання циклів, давайте переформатуємо наші дані у *довший* формат, який дозволить нам скористатися `facets` - підграфіками, кожен з яких відображає одну підмножину даних.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Набагато довше! Тепер час для деяких `ggplots`! То який `geom` ми будемо використовувати?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Легко-gg!

Тепер ми можемо побачити, що ці дані трохи шумні: спостерігаючи за кожним стовпцем у вигляді діаграми-боксплоту, ви можете помітити викиди. Ви могли б пройтися по набору даних і видалити ці викиди, але це зробило б дані досить мінімальними.

Поки що давайте оберемо, які стовпці ми будемо використовувати для нашого завдання кластеризації. Оберемо числові стовпці з подібними діапазонами. Ми могли б закодувати `artist_top_genre` як числовий, але наразі пропустимо його.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Обчислення кластеризації k-means у R

Ми можемо обчислити k-means у R за допомогою вбудованої функції `kmeans`, див. `help("kmeans()")`. Функція `kmeans()` приймає датафрейм з усіма числовими стовпцями як основний аргумент.

Першим кроком при використанні кластеризації k-means є визначення кількості кластерів (k), які будуть створені у фінальному рішенні. Ми знаємо, що в наборі даних є 3 музичні жанри, які ми виділили, тому спробуємо 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


Об'єкт kmeans містить кілька частин інформації, які добре пояснені у `help("kmeans()")`. Зараз зосередимося на кількох аспектах. Ми бачимо, що дані були згруповані у 3 кластери розміром 65, 110, 111. Вихідні дані також містять центри кластерів (середні значення) для 3 груп за 5 змінними.

Вектор кластеризації — це призначення кластерів для кожного спостереження. Давайте використаємо функцію `augment`, щоб додати призначення кластерів до початкового набору даних.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Чудово, ми щойно розділили наш набір даних на 3 групи. Тож, наскільки добре виконано кластеризацію 🤷? Давайте поглянемо на `Silhouette score`.

### **Silhouette score**

[Аналіз силуету](https://en.wikipedia.org/wiki/Silhouette_(clustering)) можна використовувати для вивчення відстані між отриманими кластерами. Цей показник варіюється від -1 до 1, і якщо значення близьке до 1, кластер щільний і добре відокремлений від інших кластерів. Значення, близьке до 0, вказує на кластеризацію з перекриттям, де зразки дуже близькі до межі прийняття рішення сусідніх кластерів. [джерело](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Метод середнього силуету обчислює середній силует спостережень для різних значень *k*. Високий середній показник силуету свідчить про якісну кластеризацію.

Функція `silhouette` у пакеті кластерів використовується для обчислення середньої ширини силуету.

> Силует можна обчислити за допомогою будь-якої [метрики відстані](https://en.wikipedia.org/wiki/Distance "Distance"), наприклад, [евклідової відстані](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") або [мангеттенської відстані](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance"), які ми обговорювали в [попередньому уроці](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Наш результат становить **.549**, тобто прямо посередині. Це свідчить про те, що наші дані не надто добре підходять для такого типу кластеризації. Давайте перевіримо цю здогадку візуально. Пакет [factoextra](https://rpkgs.datanovia.com/factoextra/index.html) надає функції (`fviz_cluster()`) для візуалізації кластеризації.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Перекриття кластерів свідчить про те, що наші дані не надто добре підходять для такого типу кластеризації, але давайте продовжимо.

## 4. Визначення оптимальної кількості кластерів

Основне питання, яке часто виникає при кластеризації методом K-Means, звучить так: без відомих міток класів, як визначити, на скільки кластерів розділити дані?

Один із способів дізнатися це — використати вибірку даних для `створення серії моделей кластеризації` з поступовим збільшенням кількості кластерів (наприклад, від 1 до 10) і оцінити метрики кластеризації, такі як **Silhouette score.**

Давайте визначимо оптимальну кількість кластерів, обчисливши алгоритм кластеризації для різних значень *k* і оцінюючи **Суму квадратів всередині кластерів** (WCSS). Загальна сума квадратів всередині кластерів (WCSS) вимірює компактність кластеризації, і ми прагнемо, щоб вона була якомога меншою, оскільки менші значення означають, що точки даних ближчі одна до одної.

Давайте дослідимо вплив різних значень `k`, від 1 до 10, на цю кластеризацію.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Тепер, коли ми маємо загальну суму квадратів всередині кластерів (tot.withinss) для кожного алгоритму кластеризації з центром *k*, ми використовуємо [метод ліктя](https://en.wikipedia.org/wiki/Elbow_method_(clustering)), щоб знайти оптимальну кількість кластерів. Цей метод полягає у побудові графіка WCSS як функції кількості кластерів і виборі [точки ліктя кривої](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") як кількості кластерів для використання.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


Графік показує значне зменшення WCSS (тобто більшу *щільність*) при збільшенні кількості кластерів з одного до двох, а також подальше помітне зменшення з двох до трьох кластерів. Після цього зменшення стає менш вираженим, що призводить до утворення `згину` 💪 на графіку приблизно на трьох кластерах. Це є хорошим свідченням того, що існує два-три досить добре розділені кластери точок даних.

Тепер ми можемо перейти до витягування моделі кластеризації, де `k = 3`:

> `pull()`: використовується для витягування одного стовпця
>
> `pluck()`: використовується для індексації структур даних, таких як списки


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Чудово! Давайте візуалізуємо отримані кластери. Хочете додати трохи інтерактивності за допомогою `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Можливо, ми очікували б, що кожен кластер (позначений різними кольорами) матиме чітко визначені жанри (позначені різними формами).

Давайте розглянемо точність моделі.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Точність цієї моделі непогана, але не ідеальна. Можливо, дані не дуже добре підходять для кластеризації методом K-Means. Ці дані надто незбалансовані, мало корельовані, і між значеннями стовпців занадто велика варіація, щоб добре кластеризувати. Насправді, кластери, які утворюються, ймовірно, сильно впливаються або спотворюються трьома категоріями жанрів, які ми визначили вище.

Проте, це був досить цікавий процес навчання!

У документації Scikit-learn можна побачити, що модель, як ця, з кластерами, які не дуже чітко розмежовані, має проблему "варіації":

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Інфографіка з Scikit-learn</figcaption>



## **Варіація**

Варіація визначається як "середнє квадратів різниць від середнього значення" [джерело](https://www.mathsisfun.com/data/standard-deviation.html). У контексті цієї проблеми кластеризації це означає, що дані нашого набору мають тенденцію занадто сильно відхилятися від середнього значення.

✅ Це чудовий момент, щоб подумати про всі способи, якими можна виправити цю проблему. Трохи змінити дані? Використати інші стовпці? Застосувати інший алгоритм? Підказка: Спробуйте [масштабувати дані](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/), щоб нормалізувати їх і протестувати інші стовпці.

> Спробуйте цей '[калькулятор варіації](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)', щоб краще зрозуміти концепцію.

------------------------------------------------------------------------

## **🚀Виклик**

Приділіть трохи часу цьому ноутбуку, змінюючи параметри. Чи можете ви покращити точність моделі, краще очистивши дані (наприклад, видаливши аномалії)? Ви можете використовувати ваги, щоб надати більшу вагу певним зразкам даних. Що ще можна зробити, щоб створити кращі кластери?

Підказка: Спробуйте масштабувати дані. У ноутбуці є закоментований код, який додає стандартне масштабування, щоб зробити стовпці даних більш схожими за діапазоном. Ви побачите, що хоча оцінка силуету знижується, "злам" на графіку ліктя згладжується. Це тому, що залишення даних без масштабування дозволяє даним з меншою варіацією мати більшу вагу. Прочитайте більше про цю проблему [тут](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Тест після лекції**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Огляд і самостійне навчання**

-   Ознайомтеся з симулятором K-Means [наприклад, таким](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Ви можете використовувати цей інструмент для візуалізації зразків даних і визначення їхніх центроїдів. Ви можете редагувати випадковість даних, кількість кластерів і кількість центроїдів. Чи допомагає це вам зрозуміти, як дані можуть бути згруповані?

-   Також ознайомтеся з [цією пам'яткою про K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) зі Стенфорда.

Хочете спробувати свої нові навички кластеризації на наборах даних, які добре підходять для кластеризації методом K-Means? Дивіться:

-   [Навчання і оцінка моделей кластеризації](https://rpubs.com/eR_ic/clustering) за допомогою Tidymodels та інших інструментів

-   [Аналіз кластерів методом K-Means](https://uc-r.github.io/kmeans_clustering), UC Business Analytics R Programming Guide

- [Кластеризація методом K-Means з принципами tidy data](https://www.tidymodels.org/learn/statistics/k-means/)

## **Завдання**

[Спробуйте різні методи кластеризації](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## ДЯКУЄМО:

[Jen Looper](https://www.twitter.com/jenlooper) за створення оригінальної версії цього модуля на Python ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) за створення дивовижних ілюстрацій, які роблять R більш привітним і захоплюючим. Знайдіть більше ілюстрацій у її [галереї](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Щасливого навчання,

[Eric](https://twitter.com/ericntay), Золотий студентський посол Microsoft Learn.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Ілюстрація від @allison_horst</figcaption>



---

**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, зверніть увагу, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ мовою оригіналу слід вважати авторитетним джерелом. Для критично важливої інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникли внаслідок використання цього перекладу.
